# OSU Electricity Usage by Building and Month

Builds a table: **rows** = building name (`sitename`), **columns** = month, **values** = total electricity usage (kWh).

Requires the OSU energy data under `data/advanced_core/` and `data/advanced_bonus/` (run **OSU_Energy_Dataset_Starter_Pandas.ipynb** first if needed).

In [2]:
import os
import pandas as pd

# Path to data (same as starter: project data folder locally)
if os.environ.get("DATABRICKS_RUNTIME_VERSION"):
    workspace_folder = "/Workspace/Users/anshumanr10@gmail.com/"
else:
    workspace_folder = os.path.join(os.getcwd(), "data")

def get_meter_csv_paths():
    paths = []
    for sub in ("advanced_core", "advanced_bonus"):
        d = os.path.join(workspace_folder, sub)
        if not os.path.isdir(d):
            continue
        for f in os.listdir(d):
            if f.startswith("meter-readings-") and f.endswith(".csv"):
                paths.append(os.path.join(d, f))
    return sorted(paths)

csv_paths = get_meter_csv_paths()
print(f"Found {len(csv_paths)} meter-reading CSV files.")
if not csv_paths:
    raise FileNotFoundError("No meter-readings-*.csv under data/advanced_core or data/advanced_bonus. Run OSU_Energy_Dataset_Starter_Pandas.ipynb first.")

Found 12 meter-reading CSV files.


In [3]:
# Process one file at a time: aggregate in memory, then discard raw data (avoids loading all CSVs at once)
monthly_chunks = []
for path in csv_paths:
    df = pd.read_csv(path, encoding="latin1")
    df = df[df["utility"] == "ELECTRICITY"].copy()
    if df.empty:
        continue
    df["month"] = pd.to_datetime(df["readingtime"], errors="coerce").dt.to_period("M")
    df = df.dropna(subset=["month", "readingvalue"])
    # Aggregate to one row per (sitename, month) for this file, then keep only that small result
    chunk = df.groupby(["sitename", "month"], as_index=False)["readingvalue"].sum()
    chunk["month_str"] = chunk["month"].astype(str)
    monthly_chunks.append(chunk[["sitename", "month_str", "readingvalue"]])
    del df  # free memory before next file

monthly = pd.concat(monthly_chunks, ignore_index=True)
# Sum in case same (sitename, month) appears in multiple files
monthly = monthly.groupby(["sitename", "month_str"], as_index=False)["readingvalue"].sum()

In [4]:
# Pivot: building name (row) x month (column) = total electricity (kWh)
building_month_table = monthly.pivot(
    index="sitename",
    columns="month_str",
    values="readingvalue"
).fillna(0)

# Sort columns chronologically
building_month_table = building_month_table.reindex(
    sorted(building_month_table.columns, key=lambda x: (x[:4], x[5:7])),
    axis=1
)

building_month_table

month_str,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,2026-01
sitename,,,,,,,,,,,,,
"11th Ave, 33 W",5678.033035,5903.752635,7354.725473,6925.192930,5627.295316,7300.977466,8754.954593,7591.814725,6544.611202,7408.279717,5919.309137,5888.758679,0.000000
"11th Ave, 45 W",250.177750,195.692877,162.186452,122.901199,146.021427,663.209839,951.697389,608.276839,412.431478,351.207864,275.204309,343.020414,2.707548
"11th Ave, 53 W",166.822071,145.755431,151.340137,244.358342,287.649669,525.868813,687.549059,583.096300,432.012802,276.689067,181.471180,208.287444,1.405138
"12th Ave, 395 W",38655.859614,34017.538978,41917.533875,46514.116535,53924.779927,61678.579210,68758.522036,60602.763809,56853.311751,54773.871254,44500.216109,39277.713301,205.264769
"18th Ave, 209 W",596.234179,560.041606,572.159683,585.159448,467.716555,448.202663,356.656970,317.600415,615.975161,578.303086,584.947966,551.957254,3.160956
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wiseman Hall,59068.714656,55525.022379,68679.571656,75519.785648,91436.811474,115447.062414,126960.894297,116058.095673,104933.461936,85532.359135,67582.683533,60417.338543,220.349188
Women's Field House,8219.877418,4703.624032,4185.545678,3602.011153,1077.419276,1134.576377,1392.114808,3072.255958,7716.384317,8906.975925,9607.081526,6805.068258,18.304233
Woody Hayes Athletic Center,57463.150086,54091.419451,64561.931832,69091.558974,70915.718148,89722.146414,98299.616884,92680.809643,82231.097927,75987.616250,63983.553610,61228.007295,302.147141
